In [1]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers, regularizers
from keras.layers import *

import tensorflow as tf

from keras import backend as K

# Auxilary

# Model

## Building block

### Resnet

In [2]:
def resnet_unit(feat_dim, kernel_size, x_in):
    # conv = Conv2D(feats, kernel, padding="same")
    res = keras.Sequential([
        Conv2D(feat_dim,
               kernel_size, padding="same",
               kernel_initializer='he_uniform',
               bias_initializer='he_uniform'),
        ReLU(),
        Conv2D(feat_dim,
               kernel_size,
               padding="same",
               kernel_initializer='he_uniform',
            bias_initializer='he_uniform')
    ])
    return ReLU()(x_in + res(x_in))

def resnet_block(x_in, feat_dim, kernel_size, reps, pooling = True):
    # Stage 2
    conv1 = Conv2D(feat_dim,
                   kernel_size,
                   padding="same",
                   kernel_initializer='he_uniform',
                   bias_initializer='he_uniform')(x_in)
    relu1 = ReLU()(conv1)
    conv2 = Conv2D(feat_dim,
                   kernel_size,
                   padding="same",
                   kernel_initializer='he_uniform',
                   bias_initializer='he_uniform')(relu1)
    x = ReLU()(conv2)
    for _ in range(reps):
        x = resnet_unit(feat_dim,kernel_size,x)
    if pooling == True:
        x = MaxPooling2D(2,2)(x)
        return x
    else:
        return x

### Convolution block

In [3]:
def conv_unit(feat_dim, kernel_size, x):
    x = Conv2D(feat_dim, 
               kernel_size, 
               activation = LeakyReLU(0.2), 
               padding="same", 
               kernel_initializer='he_uniform', 
               bias_initializer='he_uniform')(x)
    x = Conv2D(feat_dim,
               1,
               activation = LeakyReLU(0.2),
               padding="same",
               kernel_initializer='he_uniform',
               bias_initializer='he_uniform')(x)
    return x

def conv_block_down(x, feat_dim, reps, kernel_size, mode = 'normal'):
    if mode == 'down':
        x = MaxPooling2D(2,2)(x)
    for _ in range(reps):
        x = conv_unit(feat_dim, 
                      kernel_size,
                      x)

    return x

def conv_block_up_w_concat(x, x1, feat_dim, reps, kernel_size, mode = 'normal'):
    if mode == 'up':
        x = UpSampling2D((2,2))(x)
    
    x = Concatenate()([x,x1])
    for _ in range(reps):
        x = conv_unit(feat_dim,
                      kernel_size,
                      x)
    return x

def conv_block_up_wo_concat(x, feat_dim, reps, kernel_size, mode = 'normal'):
    if mode == 'up':
        x = UpSampling2D((2,2))(x)
    for _ in range(reps):
        x = conv_unit(feat_dim,
                      kernel_size,
                      x)
    return x

### SPADE

In [6]:
class SPADE(layers.Layer):
    def __init__(self, filters, epsilon=1e-5, **kwargs):
        super().__init__(**kwargs)
        self.epsilon = epsilon
        self.conv = layers.Conv2D(filters, 3, padding="same", activation="relu",kernel_initializer='he_uniform',
               bias_initializer='he_uniform',kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))
        self.conv_gamma = layers.Conv2D(filters, 3, padding="same",kernel_initializer='he_uniform',
               bias_initializer='he_uniform',kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))
        self.conv_beta = layers.Conv2D(filters, 3, padding="same",kernel_initializer='he_uniform',
               bias_initializer='he_uniform',kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))

    def build(self, input_shape):
        self.resize_shape = input_shape[1:3]
        # print(self.resize_shape)

    def call(self, input_tensor, raw_mask):
        mask = tf.image.resize(raw_mask, self.resize_shape, method="nearest")
        x = self.conv(mask)    

        gamma = self.conv_gamma(x)
        beta = self.conv_beta(x)
        mean, var = tf.nn.moments(input_tensor, axes=(0, 1, 2), keepdims=True)
        std = tf.sqrt(var + self.epsilon)
        # print(mean.shape)

        normalized = (input_tensor - mean) / std
        output = gamma * normalized + beta

        return output

def spade_generator_unit(x, mask, feats_in, kernel, upsampling = True):
    x = GaussianNoise(0.05)(x)
    # SPADE & conv
    spade1 = SPADE(feats_in)(x, mask)
    output = Conv2D(feats_in,kernel, padding='same', activation= LeakyReLU(0.2),kernel_initializer='he_uniform',
               bias_initializer='he_uniform',kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))(spade1)
    if upsampling == True:
        output = UpSampling2D(size = (2,2))(output)
        return output
    else:
        return output

### Adaptive Instance Normalization

In [4]:
class AdaIN(layers.Layer):
    def __init__(self, filters, epsilon=1e-5, **kwargs):
        super().__init__(**kwargs)
        self.epsilon = epsilon
        self.dense = layers.Dense(filters,
                                  activation = 'relu',
                                  kernel_initializer='he_uniform',
                                  bias_initializer='he_uniform')
        self.dense_gamma = layers.Dense(filters,
                                        kernel_initializer='he_uniform',
                                        bias_initializer='he_uniform')
        self.dense_beta = layers.Dense(filters,
                                       kernel_initializer='he_uniform',
                                       bias_initializer='he_uniform')

    def call(self, input_tensor, style_vector):
        x = self.dense(style_vector)
        gamma = self.dense_gamma(x)
        beta = self.dense_beta(x)
        #Normalize x[0]
        mean, var = tf.nn.moments(input_tensor, axes=(0, 1, 2), keepdims=True)
        std = tf.sqrt(var + self.epsilon)
        normalized = (input_tensor - mean) / std
        y = (x[0] - mean) / std

        #Reshape gamma and beta
        pool_shape = [-1, 1, 1, y.shape[-1]]
        gamma = tf.reshape(gamma, pool_shape) + 1.0
        beta = tf.reshape(beta, pool_shape)

        return gamma * normalized + beta

### Feature extraction CNN

In [15]:
def feature_extraction_unet(n_out_features = 128, n_base_features = 64, input_shape = (256,512,5)):
    inputs = keras.Input(shape = input_shape)

    conv1 = conv_block_down(inputs,
                            feat_dim = n_base_features,
                            reps = 1,
                            kernel_size = 3)
    conv2 = conv_block_down(conv1,
                            feat_dim = n_base_features*2,
                            reps = 1,
                            kernel_size = 3,
                            mode = 'down')
    conv3 = conv_block_down(conv2,
                            feat_dim = n_base_features*4,
                            reps = 1,
                            kernel_size = 3,
                            mode = 'down')
    conv4 = conv_block_down(conv3,
                            feat_dim = n_base_features*8,
                            reps = 1,
                            kernel_size = 3,
                            mode = 'down')
    conv5 = conv_block_down(conv4,
                            feat_dim = n_base_features*16,
                            reps = 1,
                            kernel_size = 3,
                            mode = 'down')
    conv6 = conv_block_up_wo_concat(conv5,
                                    feat_dim = n_base_features*8,
                                    reps = 1,
                                    kernel_size = 3,
                                    mode = 'up')
    
    conv7 = conv_block_up_w_concat(conv6, conv3,
                                    feat_dim = n_base_features*4,
                                    reps = 1,
                                    kernel_size = 3,
                                    mode = 'up')
    
    conv8 = conv_block_up_wo_concat(conv7,
                                    feat_dim = n_base_features*2,
                                    reps = 1,
                                    kernel_size = 3,
                                    mode = 'up')
    
    conv9 = conv_block_up_w_concat(conv8, conv1,
                                    feat_dim = n_out_features,
                                    reps = 1,
                                    kernel_size = 3,
                                    mode = 'up')

    feature_out = conv_block_up_wo_concat(conv9,
                                    feat_dim = n_out_features,
                                    reps = 1,
                                    kernel_size = 1,
                                    mode = 'normal')
    unet = keras.Model(inputs, feature_out)
    return unet

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 256, 512, 5  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_120 (Conv2D)            (None, 256, 512, 64  2944        ['input_10[0][0]']               
                                )                                                                 
                                                                                                  
 conv2d_121 (Conv2D)            (None, 256, 512, 64  4160        ['conv2d_120[0][0]']             
                                )                                                           

In [8]:
def mapping_and_recon_cnn(n_base_features = 128, input_shape = (256,512,128)):
    inputs = keras.Input(shape = input_shape)
    inputs_2 = keras.Input(shape = input_shape)
    spade1 = spade_generator_unit(inputs,
                                  inputs_2,
                                  128,
                                  3,
                                  upsampling = False)
    conv1 = resnet_block(spade1,
                         feat_dim = n_base_features,
                         kernel_size = 1,
                         reps = 2,
                         pooling = False)
    spade2 = spade_generator_unit(conv1,
                                  inputs_2,
                                  n_base_features,
                                  3,
                                  upsampling = False)
    conv2 = resnet_block(spade2,
                         feat_dim = n_base_features*2,
                         kernel_size = 1,
                         reps = 2,
                         pooling = False)
    
    spade3 = spade_generator_unit(conv2,
                                  inputs_2,
                                  n_base_features*2,
                                  3,
                                  upsampling = False)
    conv3 = resnet_block(spade3,
                         feat_dim = n_base_features*4,
                         kernel_size = 1,
                         reps = 2,
                         pooling = False)
    conv_out = Conv2D(1,1,padding='same')(conv3)
    mapping_resnet = keras.Model([inputs,inputs_2], conv3)
    return mapping_resnet
# .summary()
tf.keras.utils.plot_model(mapping_and_recon_cnn(), show_shapes=True, show_dtype=True)
# Feature extraction CNN

# Differentiator CNN

# Integrator CNN

# PARCv2

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [12]:
!pip install pydot
!pip install graphviz


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 47.0/47.0 kB 1.2 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
def integrator_cnn(n_base_features = 128, input_shape = (256,512,1)):
    inputs = keras.Input(shape = input_shape)
    conv1 = resnet_block(inputs,
                         feat_dim = n_base_features,
                         kernel_size = 1,
                         reps = 2,
                         pooling = False)
    conv2 = resnet_block(conv1,
                         feat_dim = n_base_features*2,
                         kernel_size = 1,
                         reps = 2,
                         pooling = False)
    conv3 = resnet_block(conv2,
                         feat_dim = n_base_features*4,
                         kernel_size = 1,
                         reps = 2,
                         pooling = False)
    conv_out = Conv2D(1,1,padding='same')(conv3)
    integrator_resnet = keras.Model(inputs, conv_out)
    return integrator_resnet
integrator_cnn().summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 256, 512, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_159 (Conv2D)            (None, 256, 512, 12  256         ['input_12[0][0]']               
                                8)                                                                
                                                                                                  
 re_lu_18 (ReLU)                (None, 256, 512, 12  0           ['conv2d_159[0][0]']             
                                8)                                                          

# Data pipeline

# Training

# Validation